In [1]:
import numpy as np
from sklearn import datasets
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

In [2]:
#                    Description of the dataset
#  data : {ndarray, dataframe} of shape (150, 4)
#             The data matrix. If `as_frame=True`, `data` will be a pandas
#             DataFrame.
#         target: {ndarray, Series} of shape (150,)
#             The classification target. If `as_frame=True`, `target` will be
#             a pandas Series.
#         feature_names: list
#             The names of the dataset columns.
#         target_names: list
#             The names of target classes.
#         frame: DataFrame of shape (150, 5)
#             Only present when `as_frame=True`. DataFrame with `data` and
#             `target`.

In [3]:
# Creating an output array
def target_array(raw_target):
    target = []
    for i in range(len(raw_target)):
        if raw_target[i] == 0:
            target.append([1, 0, 0])
        elif raw_target[i] == 1:
            target.append([0, 1, 0])
        else:
            target.append([0, 0, 1])
    return np.array(target, dtype="uint8")
    

In [4]:
# Loading the Iris dataset
iris = datasets.load_iris()

data = iris.data
target = target_array(iris.target)

# Mixing data
rs1 = RandomState(MT19937(SeedSequence(0)))
rs1.shuffle(data)
rs2 = RandomState(MT19937(SeedSequence(0)))
rs2.shuffle(target)

In [5]:

# Sigmoid 
# Activation function
def activation_func(x):
    z = 1 / (1 + (np.e ** -x))
    return z


# Derivative of the activation function
def deriv_activation_func(x):
    z = activation_func(x)
    return z * (1 - z)
    

0 - input layer  
1 - hidden layer  
2 - output layer  
weights_1_to_2 - Weights linking the hidden layer and the output layer

In [6]:
# Neural network learning function
def neural_network_training(weights_0_to_1,
                            weights_1_to_2,
                            data,
                            target,
                            number_training_data,
                            epochs=5,
                            alpha=0.5):
    epoch_print_step = 100
    for epoch in range(epochs):
        if epoch % epoch_print_step == 0:
            print(f"\nepoch: {epoch + 1}")
        for i in range(number_training_data):
            # Forward propagation
            layer_0 = data[i:i + 1]
            layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
            layer_2 = np.dot(layer_1, weights_1_to_2)

            # Error calculation
            layer_2_delta = layer_2 - target[i:i + 1]
            layer_1_delta = layer_2_delta.dot(weights_1_to_2.T) * deriv_activation_func(layer_1)
            
            # Backpropagation
            weights_1_to_2 -= alpha * layer_1.T.dot(layer_2_delta)
            weights_0_to_1 -= alpha * layer_0.T.dot(layer_1_delta)
            
            # Output of a part of the data
            if epoch % epoch_print_step == 0 and i >= 10 and i <= 13:
                print(f"{np.argmax(layer_2) == np.argmax(target[i])} Error: {np.sum((layer_2 - target[i]) ** 2):.5f}, out = {layer_2}, target = {target[i]}")
  
    return weights_0_to_1, weights_1_to_2

In [7]:
# Declaring parameters for a neural network
# epochs = 1000
epochs = 401
alpha = 0.002

number_test_data = 30
len_data = len(data)
number_training_data = len_data - number_test_data
layer_1_size = 6

np.random.seed(1)
weights_0_to_1 = np.random.rand(len(data[0]), layer_1_size)
weights_1_to_2 = np.random.rand(layer_1_size, len(target[0]))

In [8]:
# Neural network training
weights_0_to_1, weights_1_to_2 = neural_network_training(weights_0_to_1,
                                                         weights_1_to_2,
                                                         data,
                                                         target,
                                                         number_training_data,
                                                         epochs,
                                                         alpha)


epoch: 1
True Error: 18.61402, out = [[2.58528398 3.40121715 2.48283695]], target = [0 1 0]
True Error: 18.59745, out = [[2.58266427 3.40825556 2.47539982]], target = [0 1 0]
True Error: 18.10447, out = [[2.54412668 3.37217269 2.45044694]], target = [0 1 0]
False Error: 16.97011, out = [[2.36283    3.13781248 2.29498066]], target = [1 0 0]

epoch: 101
False Error: 0.67049, out = [[0.24230168 0.30095862 0.35089045]], target = [0 1 0]
False Error: 0.76293, out = [[0.10477757 0.3371258  0.55906148]], target = [0 1 0]
False Error: 0.63567, out = [[0.24874453 0.32427229 0.34232744]], target = [0 1 0]
True Error: 0.17764, out = [[ 0.74120654  0.30239109 -0.13863862]], target = [1 0 0]

epoch: 201
True Error: 0.59450, out = [[0.2636368  0.34764159 0.31531339]], target = [0 1 0]
False Error: 0.73703, out = [[0.07592499 0.35240744 0.55847426]], target = [0 1 0]
True Error: 0.55835, out = [[0.26765489 0.37292896 0.30577012]], target = [0 1 0]
True Error: 0.12819, out = [[ 0.82808844  0.28245305

In [9]:
def сhecking_accuracy(data, target):    
    if np.argmax(data) == np.argmax(target):
        print("True")
        return
    print(f"False: {data}, {target}")

In [10]:
def neural_network_test(weights_0_to_1,
                        weights_1_to_2,
                        data,
                        target,
                        number_test_data,
                        len_data):
    # print(len_data - number_test_data, len_data)
    for i in range(len_data - number_test_data, len_data):
        layer_0 = data[i:i + 1]
        layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
        layer_2 = np.dot(layer_1, weights_1_to_2)
        # error = np.sum((layer_2 - target[i]) ** 2)
        
        сhecking_accuracy(layer_2, target[i])
        # print(f"target = out: {np.argmax(target[i]) == np.argmax(layer_2)}\terror = {error:.4f}")
        # print(f"target = {target[i]}, out = {layer_2},\terror = {error:.4f}")

In [11]:
neural_network_test(weights_0_to_1,
                    weights_1_to_2,
                    data,
                    target,
                    number_test_data,
                    len_data)

False: [[-0.05238533  0.30844582  0.75063758]], [0 1 0]
True
False: [[-0.03617612  0.26725785  0.74857704]], [0 1 0]
True
True
True
True
True
True
True
True
True
False: [[0.08054389 0.4570365  0.45931611]], [0 1 0]
True
True
True
True
True
True
True
True
True
True
True
True
True
False: [[0.00501959 0.29442758 0.67119396]], [0 1 0]
True
True
True


In [12]:
def neural_network(weights_0_to_1,
                   weights_1_to_2,
                   data):
    layer_0 = data
    layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
    layer_2 = np.dot(layer_1, weights_1_to_2)    
    return layer_2

In [13]:
for i in range(20):
    number = np.random.randint(0, 150)
    prediction = neural_network(weights_0_to_1,
                                weights_1_to_2,
                                data[number])
    сhecking_accuracy(prediction, target[number])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
